<a href="https://colab.research.google.com/github/Vaish181204/Suvidha-Foundation/blob/main/DatasetCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U transformers datasets accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


In [2]:
pip install rouge-score bert-score evaluate pandas openpyxl tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1d3742f99998ed44e16730796f1790739423e4bdc964e9d69f076057ab637b54
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [3]:
import torch
print(f"Is CUDA available: {torch.cuda.is_available()}")
print(f"Device Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU found'}")

Is CUDA available: True
Device Name: Tesla T4


In [4]:
import pandas as pd
import os

In [7]:
file_path = '/content/NewsSumm Dataset.xlsx'

In [8]:
if os.path.exists(file_path):
    df = pd.read_excel(file_path)
    print(df.columns)
    df = df.dropna(subset=['article_text', 'human_summary'])
    pilot_df =df.sample(n=1000, random_state=42)
    pilot_df.to_csv('newsum.csv', index=False)
    print(f"Sucess!!! Pilot dataset created with {len(pilot_df)} samples.")
    print(pilot_df.head())
else:
    print("Oppss... File not found. Please check the path of your NewsSum download.")
    print(pilot_df.head())

Index(['newspaper_name', 'published_date\n', 'headline', 'article_text',
       'human_summary', 'news_category'],
      dtype='object')
Sucess!!! Pilot dataset created with 1000 samples.
               newspaper_name     published_date\n  \
196658        Hindustan Times  2022-03-03 00:00:00   
131231         Goa Chronicle   2020-07-22 00:00:00   
291863       Arunachal Front   2020-12-04 00:00:00   
94989   The Imphal free press  2023-03-28 00:00:00   
189518     The Times of India  2024-04-01 00:00:00   

                                                 headline  \
196658                             City reports 100 cases   
131231                FabCoders decodes Appsolutely right   
291863  Itanagar: Arunachal Cricket Association’s AGM ...   
94989   Urea allocation will be reduced to encourage n...   
189518  State to seek ECI approval to announce new DSC...   

                                             article_text  \
196658  Mumbai : The city continued to report low case...  

In [9]:
import re
import html

In [10]:
df = pilot_df.copy()

In [11]:
def clean_for_research(text):
    if not isinstance(text, str): return ""
    text = html.unescape(text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [12]:
df['article_text'] = df['article_text'].apply(clean_for_research)
df['human_summary'] = df['human_summary'].apply(clean_for_research)

In [13]:
df = df[(df['article_text'].str.len() > 100) & (df['human_summary'].str.len() > 20)]

In [14]:
df.to_csv('newsum_ready.csv', index=False)
print("Great Work!!! The data file 'newsum_ready.csv' is good to go!!!")

Great Work!!! The data file 'newsum_ready.csv' is good to go!!!


In [15]:
def professional_clean(text):
    if not isinstance(text, str): return ""
    text = html.unescape(text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = text.replace('\xa0', ' ').replace('\xad', '')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [16]:
df['article_text'] = df['article_text'].apply(professional_clean)
df['human_summary'] = df['human_summary'].apply(professional_clean)

In [17]:
df['article_word_count'] = df['article_text'].apply(lambda x: len(x.split()))
df['summary_word_count'] = df['human_summary'].apply(lambda x: len(x.split()))

In [18]:
output_excel = 'NewsSumm_Final_Cleaned.xlsx'
df.to_excel(output_excel, index=False)

In [19]:
df.to_csv('newsum_ready.csv', index=False)

In [20]:
print("\n" + "="*45)
print("     OFFICIAL DATA PREPROCESSING REPORT")
print("="*45)
print(f"Final Dataset Size: {len(df)} samples")
print(f"Avg Article Length: {df['article_word_count'].mean():.1f} words")
print(f"Avg Summary Length: {df['summary_word_count'].mean():.1f} words")
print("-" * 45)
print("Cleaning Success: HTML decoded, URLs removed, Source tags stripped.")
print(f"Files Generated: {output_excel} and 'newsum_ready.csv'")
print("="*45)


     OFFICIAL DATA PREPROCESSING REPORT
Final Dataset Size: 993 samples
Avg Article Length: 335.9 words
Avg Summary Length: 95.4 words
---------------------------------------------
Cleaning Success: HTML decoded, URLs removed, Source tags stripped.
Files Generated: NewsSumm_Final_Cleaned.xlsx and 'newsum_ready.csv'
